In [1]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth
import random
import re

import pandas as pd
from bs4 import BeautifulSoup
import requests

from time import sleep
import random
print('Finish import package')
sleep(1)

Finish import package


In [2]:
import cloudscraper

scraper = cloudscraper.create_scraper()

# Make a request to a website that is protected by Cloudflare.
response = scraper.get("https://www.sciencedirect.com/browse/journals-and-books?contentType=JL")

# Check if the response is a Cloudflare challenge page.
if response.status_code == 503:
  print("Cloudflare challenge detected.")
else:
  print("Cloudflare not detected.")

sleep(4)

Cloudflare not detected.


In [3]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--incognito")

options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_experimental_option('useAutomationExtension', False)

# Add argument with proxy infos:
# port = config.rand_port()

web = "https://www.sciencedirect.com/browse/journals-and-books?contentType=JL"
path = "C:/WebDriver/chromedriver.exe"


driver_service = Service(executable_path=path)
driver = webdriver.Chrome(service=driver_service) 

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get(web) 
print('>>>>  Get link Successfully  >>>>')
sleep(3)

>>>>  Get link Successfully  >>>>


In [4]:
list_names = []
list_links = []
list_ISSNs = []
list_Impact_Factors = []

In [5]:
for page in range(47,50):
    driver.get(f"https://www.sciencedirect.com/browse/journals-and-books?page={page}&contentType=JL")
    page_source = BeautifulSoup(driver.page_source)
    print('Get data - page:',page)

    magazine_names = page_source.find_all(name='a',attrs={'class':'anchor js-publication-title anchor-default'})
    for name in magazine_names:
        a = name.find(name='span', attrs = {'class':'anchor-text'})
        list_names.append(a.text.strip())
        list_links.append('https://www.sciencedirect.com'+ name['href'])
    print(f'Page {page} --  Done')
    sleep(1)

Get data - page: 47
Page 47 --  Done
Get data - page: 48
Page 48 --  Done
Get data - page: 49
Page 49 --  Done


In [21]:
len(list_names)

216


In [22]:
len(list_links)

216


In [8]:
for i in list_links[:]: 
    driver.get(i)
    page_source = BeautifulSoup(driver.page_source)

    # ISSN  
    elems_scores = page_source.find(name='div', attrs={'class':'js-issn text-s'})
    # print(elems_score)
    elems = [elems_score.text for elems_score in elems_scores]
    list_ISSNs.append(elems)

    # Impactors
    try:
        imp = driver.find_element('xpath','//*[@id="react-root"]/div/div/div/main/div[1]/div/section/div[1]/div/div[2]/div[2]/div[2]/button/span/span[1]').text
        list_Impact_Factors.append(imp)
    except:
        list_Impact_Factors.append("")
    sleep(1)

In [23]:
len(list_ISSNs)

216


In [11]:
# Khởi tạo danh sách kết quả
issn_list = []
online_issn_list = []
linking_issn_list = []
print_issn_list = []

In [12]:
# Biểu thức chính quy để trích xuất phần sau của "ISSN", "Online ISSN", "Linking ISSN", "Print ISSN"
pattern = r'((ISSN|Online ISSN|Linking ISSN|Print ISSN): (\d{4}-\d{4}))'

In [13]:
# Xử lý danh sách
for sublist in list_ISSNs:
    issn = ""
    online_issn = ""
    linking_issn = ""
    print_issn = ""
    
    for item in sublist:
        match = re.search(pattern, item)
        if match:
            if match.group(2) == "ISSN":
                issn = match.group(3)
            elif match.group(2) == "Online ISSN":
                online_issn = match.group(3)
            elif match.group(2) == "Linking ISSN":
                linking_issn = match.group(3)
            elif match.group(2) == "Print ISSN":
                print_issn = match.group(3)
    
    issn_list.append(issn)
    online_issn_list.append(online_issn)
    linking_issn_list.append(linking_issn)
    print_issn_list.append(print_issn)

In [24]:
print(f"ISSN: {len(issn_list)}")
print(f"Online ISSN: {len(online_issn_list)}")
print(f"Linking ISSN: {len(linking_issn_list)}")
print(f"Print ISSN: {len(print_issn_list)}")
sleep(2)

ISSN: 216
Online ISSN: 216
Linking ISSN: 216
Print ISSN: 216


In [25]:
len(list_Impact_Factors[:])

216


In [17]:
driver.quit()

In [18]:
data = pd.DataFrame({
    'Tên tạp chí' : list_names,
    'ISSN' : issn_list,
    'eISSN': online_issn_list,
    'Linking ISSN': linking_issn_list,
    'Print ISSN': print_issn_list,
    'Impact Factor': list_Impact_Factors
})

In [ ]:
# excel_path = r"C:\Project's Cong\HKDN UEH\data.xlsx"

# data.to_excel(excel_path, index=False)

In [19]:
import xlwings as xw

#--------------------------------------------------
# ADJUST THE FOLLOWING:
wb_name = "C:\Project's Cong\HKDN UEH\data.xlsx"
sheet_name = "Sheet1"
df_mapping = {"A4602": data}
#--------------------------------------------------

# Open Excel in background
with xw.App(visible=False) as app:
    wb = app.books.open(wb_name)

    # Add sheet if it does not exist
    current_sheets = [sheet.name for sheet in wb.sheets]
    if sheet_name not in current_sheets:
        wb.sheets.add(sheet_name)

    # Write dataframe to cell range
    for cell_target, df in df_mapping.items():
        wb.sheets(sheet_name).range(cell_target).options(pd.DataFrame, index=False).value = df

    wb.save()